In [1]:
import numpy as np
import WLCgreen as wlc
import matplotlib.pyplot as plt
%matplotlib
from numba import jit

Using matplotlib backend: Qt5Agg


# Eigenvalues

In [2]:
npts=100
Kvals=np.logspace(-1,5,npts)
ORDEig=30
allEvals=np.zeros((npts,ORDEig),dtype='complex')
mu=0
for ii in range(0,npts):
    allEvals[ii,:]= wlc.wlc_eigvals(Kvals[ii],ORDEig,mu)  
    
for ii in range(mu,ORDEig):
    plt.semilogx(Kvals,np.real(allEvals[:,ii]))
    
plt.figure()
plt.ylim([-250,0])
plt.show()
for ii in range(mu,ORDEig):
    plt.semilogx(Kvals,np.imag(allEvals[:,ii]))
plt.ylim([-250,250])
plt.show()  

# G and derivatives

In [3]:
mu=2
K=1.0
lamMax=500
lamNeed=10
lower=-25

npts=5000
pp=np.linspace(lower,0.01,npts)
out=np.zeros([npts,5],dtype='complex')
out2=np.zeros([npts,3],dtype='complex')
for ii in range(0,npts):
    p=pp[ii]
    jp=wlc.get_jp(p,mu,K,lamMax)
    jm= wlc.get_jm(p,mu,K,lamMax)
    djp=wlc.get_djp(p,mu,K,lamMax,jp)
    djm=wlc.get_djm(p,mu,K,lamMax,jm)
    W=wlc.get_W(p,mu,K,lamMax,jp,jm)
    dwm=wlc.get_dw(jm,djm,mu)
    dwp=wlc.get_dw(jp,djp,mu)
    dW=wlc.get_dW(p,mu,K,lamMax,dwp,dwm,W)
    
    lam=2
    lam0=2
    out[ii,0]=wlc.InvGprimeGeneral(p,lam,lam0,mu,K,jp,jm,djp,djm)
    out[ii,1]=wlc.Xp(lam,lam0,jp)
    out[ii,2]=wlc.X1(p,lam0,mu,K,jp,jm)*wlc.Xpprime(lam,lam0,jp,djp,out[ii,1])
    out[ii,3]=wlc.get_G(p,lam,lam0,mu,K,jm,jp,W)
    out[ii,4]=wlc.InvG(p,lam,lam0,mu,K,jp,jm)
    
    G=wlc.get_G_zero(lam,lam0,mu,K,jm,jp,W)
    dG=wlc.get_dG_zero(lam,lam0,mu,K,jm,jp,W,dW,dwm,dwp)
    dG=wlc.get_dG(p,lam,lam0,mu,K,jm,jp,W,dW,dwm,dwp)

ORDEig=100
eigvals = wlc.wlc_eigvals(K,ORDEig,mu,d=3)

In [4]:
fig, ax = plt.subplots()
ax.plot(pp,out[:,0].real,label='d(1/G)/dp')
#ax.plot(pp,out[:,1].real,label='Xp')
#ax.plot(pp,out[:,2].real,label='X1*dXp')
#ax.plot(pp,out[:,3].real,label='G')
ax.plot(pp,out[:,4].real,label='1/G')
ax.plot(eigvals.real,eigvals.real*0.0,'o',label='eigval')
ax.legend()
ax.set_xlabel('p')
ax.set_ylim([-30,30])
ax.set_xlim([lower,0])

#plt.plot(pp,out[:,0].real,label='d(1/G)/dp')
#plt.plot(pp,out[:,1].real,label='X1')
#plt.legend()
#plt.xlabel('p')
#plt.ylabel('value')
#plt.ylim([-100,100])

# using 'spines', new in Matplotlib 1.0
ax.spines['left'].set_position('zero')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('zero')
ax.spines['top'].set_color('none')
plt.show()



# Residue

In [5]:
ORDEig=100

npts=500
npoles=40
resPlot=np.zeros((npts,npoles),dtype='complex')
resSmallPlot=np.zeros((npts,npoles),dtype='complex')
resSwitch=np.zeros((npts,npoles),dtype='complex')
Ks=np.logspace(-4,4,npts)
for ii in range(0,npts):
    K=Ks[ii]
    lam0=0
    lam=0
    mu=0
    eigvals = wlc.wlc_eigvals(K,ORDEig,mu,d=3)
    for l in range(mu,npoles):
        eig=eigvals[l]
        res, resSmall = wlc.bothResidues(K,eig,l,mu,nlam=10,d=3)
        resPlot[ii,l]=res[lam0,lam]
        resSmallPlot[ii,l]=resSmall[lam0,lam]
        resSwitch[ii,l]=wlc.residues(K,eig,l,mu,nlam=10,d=3)[lam0,lam]
    


In [6]:
fig, ax = plt.subplots()
#ax.semilogx(Ks,resPlot.real,'r',label='res')  
#ax.semilogx(Ks,resSmallPlot.real,'b',label='resSmallK')
for l in range(mu,npoles):
    #ax.loglog(Ks,abs(resPlot[:,l].real),'r',label='res '+str(l))  
    #ax.loglog(Ks,Ks**0.0*abs(resSmallPlot[:,l].real),'c',label='resSmallK '+str(l))
    ax.loglog(Ks,Ks**0.0*abs(resSwitch[:,l].real),'k-',label='resSmallK '+str(l))

#ax.legend(loc=4)
ax.set_ylim([0.0000000000001,10])
ax.set_title('Re(res), lam='+str(lam)+', lam0='+str(lam0)+', mu='+str(mu))
ax.set_xlabel('K')
ax.set_ylabel('residues * K^0')
plt.show()



In [4]:
fig, ax = plt.subplots()
#ax.semilogx(Ks,resPlot.imag,'r',label='res')  
#ax.semilogx(Ks,resSmallPlot.imag,'b',label='resSmallK')
for l in range(mu,npoles):
    ax.loglog(Ks,abs(resPlot[:,l].imag),'r',label='res '+str(l))  
    ax.loglog(Ks,abs(resSmallPlot[:,l].imag),'b',label='resSmallK '+str(l))

ax.legend(loc=3)
ax.set_title('Im(res), lam='+str(lam)+', lam0='+str(lam0)+', mu='+str(mu))
ax.set_ylim([-10,10])
plt.show()